In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import mysql.connector

db = mysql.connector.connect(host = "localhost", 
                            username = "root",
                             password = "Sachin@5893",
                             database = "target"
                            )

cur = db.cursor()


# Basic Queries

**1.List all unique cities where customers are located.**

In [ ]:
query = """ select distinct customer_city from customers """
cur.execute(query)
data = cur.fetchall()
df= pd.DataFrame(data, columns= ["States"])
df.head(5)

**2.Count the number of orders placed in 2017.**

In [ ]:
query = """ select count(order_id) from orders where year(order_purchase_timestamp) = 2017 """
cur.execute(query)
data = cur.fetchall()
"Total orders placed in 2027 are ", data[0][0]

**3.Find the total sales per category.**

In [ ]:
query = """ select products.product_category category, round(sum(payments.payment_value),2) sales
from products join order_items
on products.product_id = order_items.product_id
join payments
on payments.order_id = order_items.order_id
group by category

"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns=["Category", "Sales"])
df

**4.Calculate the percentage of orders that were paid in installments.**

In [ ]:
query = """ select (sum(case when payment_installments >=1 then 1 else 0 end))/count(*)*100 from payments"""
cur.execute(query)
data = cur.fetchall()
"Percentage of orders that were paid in installments is ", data[0][0]

**5.Count the number of customers from each state**

In [ ]:
query = """ select customer_state, count(customer_id) from customers group by customer_state """
cur.execute(query)
data = cur.fetchall()
df= pd.DataFrame(data, columns=["States","No of Customers"])
#df
df = df.sort_values(by="No of Customers", ascending=False) 
plt.figure(figsize=(8,3))
plt.bar(df["States"],df["No of Customers"])
plt.xticks(rotation = 90)
plt.xlabel("States")
plt.ylabel("No of Customers")
plt.title("No of Customers by States")
plt.show()

# Intermediate Queries

**1.Calculate the number of orders per month in 2018.**

In [ ]:
query = """ select monthname(order_purchase_timestamp) months, count(order_id) order_count
from orders where year(order_purchase_timestamp) = 2018
group by months
"""
cur.execute(query)
data = cur.fetchall()
df=pd.DataFrame(data, columns= ["months","order_count"])
o = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October"]
ax = sns.barplot(x = df["months"], y = df["order_count"], data=df, color = "red", order=o)
plt.xticks(rotation = 45)
ax.bar_label(ax.containers[0])
plt.title("Count by Orders by Months in 2028")
plt.show()

**2.Find the average number of products per order, grouped by customer city.**

In [ ]:
query = """ with count_per_order as
(select orders.order_id, orders.customer_id, count(order_items.order_id) as  oc
from orders join order_items
on orders.order_id = order_items.order_id
group by orders.order_id, orders.customer_id)

select customers.customer_city, round(avg(count_per_order.oc),2) average_order
from customers join count_per_order
on customers.customer_id = count_per_order.customer_id
group by customers.customer_city order by average_order desc
    
"""
cur.execute(query)
data = cur.fetchall()
df= pd.DataFrame(data,columns=["Customer City", "Average Products/Orders"])
df.head(10)

**3.Calculate the percentage of total revenue contributed by each product category.**

In [ ]:
query = """ select upper(products.product_category) category, round((sum(payments.payment_value)/(select sum(payment_value) from payments))*100,2) sales_percentage
from products join order_items
on products.product_id = order_items.product_id
join payments
on payments.order_id = order_items.order_id
group by category order by sales_percentage desc

"""
cur.execute(query)
data = cur.fetchall()
df= pd.DataFrame(data,columns=["Category", "Percentage Distribution"])
df.head(10)

**4.Identify the correlation between product price and the number of times a product has been purchased.**

In [ ]:
query = """ select products.product_category,
count(order_items.product_id),
round(avg(order_items.price),2)
from products join  order_items
on products.product_id = order_items.product_id
group by products.product_category
"""
cur.execute(query)
data = cur.fetchall()
df= pd.DataFrame(data,columns=["Category", "order_count", "price"])

arr1= df["order_count"]
arr2 = df["price"]
a = np.corrcoef([arr1, arr2])
print("The  Correlation between price and number of times a product has been purchased", a[0][1])

**5.Calculate the total revenue generated by each seller, and rank them by revenue.**

In [ ]:
query = """ select *, dense_rank() over(order by revenue desc) as rn from
(select order_items.seller_id, sum(payments.payment_value)
revenue from order_items join payments
on order_items.order_id = payments.order_id
group by order_items.seller_id) as a
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns=["seller_id","revenue","Rank"])
df = df.head()
sns.barplot(x="seller_id", y="revenue", data=df)
plt.xticks(rotation = 90)
plt.show()

# Advanced Queries


**1. Calculate the moving average of order values for each customer over their order history.**

In [ ]:
query=""" select customer_id, order_purchase_timestamp, payment,
avg(payment) over (partition by customer_id order by order_purchase_timestamp
rows between 2 preceding and current row) as mov_avg
from
(select orders.customer_id, orders.order_purchase_timestamp,
payments.payment_value as payment
from payments join  orders
on payments.order_id = orders.order_id) as a
"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["Seller ID","Order Purchase Timestamp","Payment","Moving Average"])
df

**2. Calculate the cumulative sales per month for each year.**

In [ ]:
query=""" select years, months, payment, sum(payment)
over(order by years, months) cumulative_sales from
(select year(orders.order_purchase_timestamp) as years,
month(orders.order_purchase_timestamp) as months,
round(sum(payments.payment_value),2) as payment from orders join payments
on orders.order_id = payments.order_id
group by years, months order by years, months) as a
"""

cur.execute(query)
data = cur.fetchall()
df=pd.DataFrame(data,columns=["Years","Months","Payment","Cumulative sales"])
df

**3. Calculate the year-over-year growth rate of total sales.**

In [ ]:
query=""" with a as(select year(orders.order_purchase_timestamp) as years,
round(sum(payments.payment_value),2) as payment from orders join payments
on orders.order_id = payments.order_id
group by years order by years)

select years,((payment - lag(payment,1) over(order by years))/
lag(payment,1) over (order by years)) *  100 from a
"""

cur.execute(query)
data = cur.fetchall()
df=pd.DataFrame(data,columns=["Years","YOY% Growth"])
df

**4. Calculate the retention rate of customers, defined as the percentage of customers who make another purchase within 6 months of their first purchase.**

In [ ]:
query=""" with a as(select customers.customer_id,
min(orders.order-purchase_timestamp) first_order
from customers join orders
on customers.customer_id orders.customer_id
group by customers.customer_id),

b as(select a.customer_id, count(distinct orders.order_purchase_timestamp) next_order
from join orders
on orders.customer_id = a.customer_id
and orders.order_purchase_timestamp > first_order
and orders.order_purchase_timestamp <
date add(first_order, interval 6 month)
group by a.customer_id)

select 100 * (count(distinct a.customer_id)/ count(distinct b.customer_id))
from a left join b
on a.customer_id = b.customer_id
"""

cur.execute(query)
data = cur.fetchall()
df=pd.DataFrame(data,columns=["Years","Retention rate of customers"])
df

**5. Identify the top 3 customers who spent the most money in each year.**

In [ ]:
query=""" select years, customer_id, payment, d_rank
from
(select year(orders.order_purchase_timestamp) years,
orders.customer_id,
sum(payments.payment_value) payment,
dense_rank() over(partition by year(orders.order_purchase_timestamp)
order by sum(payments.payment_value) desc) d_rank
from orders join payments
on payments.order_id = orders.order_id
group by year(orders.order_purchase_timestamp),
orders.customer_id) as a
where d_rank<=3
"""

cur.execute(query)
data = cur.fetchall()
df=pd.DataFrame(data,columns=["Years","Customer_id","Payment", "Rank"])
sns.barplot(x="Customer_id", y="Payment",data=df, hue="Years")
plt.xticks(rotation = 90)
plt.show()